# Install or upgrade libraries

It might be that you are running with the latest libraries and that they all work together fine. 

Running the following cell takes a minute or so but ensures that you have a consistent set of python tools. 

In [ ]:
!pip install hist[plot]

!pip install --upgrade pandas

!pip install --upgrade matplotlib


## Imports

Import all the libraries we will need and check their versions, in case you run into issues. 

In [ ]:
%load_ext autoreload
%autoreload 2

# The classics
import numpy as np
import matplotlib.pylab as plt
import matplotlib # To get the version

import pandas as pd

import hist
from hist import Hist

# To read file names using the OS (operating system)
import glob
import os

In [ ]:
print("Versions --------\n")
print(f"{np.__version__ = }\n")
print(f"{matplotlib.__version__ = }\n")
print(f"{hist.__version__ = }\n")
print(f"{pd.__version__ = }\n")

### Hist

If you looked closely above, you'll see that we have a new library, 
[Hist](https://hist.readthedocs.io/en/latest/). It was developed in large part by
members of the HEP communit and will help us with some plotting we'll be doing later.

You don't *need* to use Hist and could instead use packages like [matplotlib](https://matplotlib.org/) on its own (as we will) or [seaborn](https://seaborn.pydata.org/), but Hist can make some things easier as we will see. 

# Data-driven background estimation

Even though we won't be using it with our $Z'$ search, we'll be startng with an example of *data-driven* approaches to background estimation. 

This will be a simple *side-band* estimate. In this example, there is a peak, sitting on top of some background. You can use more sophisticated approaches to estimating the background, but we will just look at how many events
are in the region *not* under the peak and then use that to estimate the number of background events *under* the peak. 

Let's start by reading in some processed data from CMS open data. These are 100,000 events where there
are two muons in the final state. We've gone ahead and calculated the combined 4-momenta for the two muons. 

We will read it in with [pandas](https://pandas.pydata.org/). 

In [ ]:
# Read in the data
data = pd.read_csv('https://raw.githubusercontent.com/mattbellis/datasets_for_education/master/physics/dimuon_summary_data.csv', names=['e','px','py','pz','charge'])


The columns are the energy and three-momenta of the combined dimuon system, as well as the charge combinations: pp is when both muons are positive, mm is when they both have negative (minus) charge, and pm is when one is positive and one is negative. 

In [ ]:
data

We can calculate the mass with some simple, relativistic kinematics. 

$$E^2 = (pc)^2 + (mc^2)^2$$

$$(mc^2)^2 = E^2 - (pc)^2$$

$$(mc^2) = \sqrt{E^2 - (pc)^2}$$

And because our units are such that we can set $c=1$...

$$m = \sqrt{E^2 - p^2}$$

In [ ]:
# Pull out the values
e = data['e'].values
px = data['px'].values
py = data['py'].values
pz = data['pz'].values

# Calculate the mass
mass = np.sqrt(e**2 - (px**2 + py**2 + pz**2))

mass

We'll plot the data around the mass of the $J/\psi$ meson. 

https://en.wikipedia.org/wiki/J/psi_meson

In [ ]:
plt.figure()
plt.hist(mass,range=(2,4),bins=100)
plt.xlabel(r'Mass of $\mu\mu$ pairs [GeV/c$^2$]',fontsize=14);

plt.tight_layout()
plt.savefig('dimuon_masses_00.png')

Let's pick out the events in the region where the peak is.

This will select both signal events that populate the peak and the background events "under" the peak, since there is no way to distinguish on an event-by-event basis which is signal and which is background. 

In the examples below, I've purposely not been very careful about identifying what is or is not the peak. You can try to tweak my values to see if you can get a better estimate. 

We'll be making use of boolean masks to pull out subsets of the data. 

In [ ]:
# Define the high/low values that pick out the peak
peak_lo = 3.0
peak_hi = 3.2

# Make a boolean mask
mass_peak_cut = (mass>peak_lo) & (mass<peak_hi)

plt.figure()
plt.hist(mass,range=(2,4),bins=100)
plt.hist(mass[mass_peak_cut],range=(2,4),bins=100, color='r')
plt.xlabel(r'Mass of $\mu\mu$ pairs [GeV/c$^2$]',fontsize=14);

plt.tight_layout()
plt.savefig('dimuon_masses_01.png');

Now let's define the sidebands on either side of the peak. 

Note that we are taking care to make sure they are the same "width" as the peak region. 

In [ ]:
# Peak boundaries
peak_lo = 3.0
peak_hi = 3.2

# Sideband boundaries
sb_left_lo = 2.8
sb_left_hi = 3.0

sb_right_lo = 3.2
sb_right_hi = 3.4

# Boolean bitmasks
mass_peak_cut = (mass>peak_lo) & (mass<peak_hi)

mass_sb_left_cut = (mass>sb_left_lo) & (mass<sb_left_hi)
mass_sb_right_cut = (mass>sb_right_lo) & (mass<sb_right_hi)

plt.figure()
plt.hist(mass,range=(2,4),bins=100)
plt.hist(mass[mass_peak_cut],range=(2,4),bins=100, color='r', label='Peak')
plt.hist(mass[mass_sb_left_cut],range=(2,4),bins=100, color='y', label='Sidebands')
plt.hist(mass[mass_sb_right_cut],range=(2,4),bins=100, color='y')

plt.xlabel(r'Mass of $\mu\mu$ pairs [GeV/c$^2$]',fontsize=14);

plt.legend()

plt.tight_layout()
plt.savefig('dimuon_masses_02.png');

We'll now count the entries in the sidebands ($N_{leftSB}$ and $N_{rightSB}$) and take the 
average ($N_{ave SB}$) as an estimate of the number of 
background events under the peak. 

We can then subtract that from the number of entries in the peak region ($N_{PR}$) to get an estimate of the
number of signal events ($N_{sig}$). 

$$N_{aveSB} = \frac{N_{leftSB} + N_{rightSB}}{2}$$

$$N_{sig} = N_{PR} - N_{ave SB}$$

In [ ]:
# Boundaries
peak_lo = 3.0
peak_hi = 3.2

sb_left_lo = 2.8
sb_left_hi = 3.0

sb_right_lo = 3.2
sb_right_hi = 3.4

# Cuts
mass_peak_cut = (mass>peak_lo) & (mass<peak_hi)

mass_sb_left_cut = (mass>sb_left_lo) & (mass<sb_left_hi)
mass_sb_right_cut = (mass>sb_right_lo) & (mass<sb_right_hi)

# Count the events
N_peak = len(mass[mass_peak_cut])
N_sb_left = len(mass[mass_sb_left_cut])
N_sb_right = len(mass[mass_sb_right_cut])

print(f"N in peak region:    {N_peak}")
print(f"N in left sideband:  {N_sb_left}")
print(f"N in right sideband: {N_sb_right}")
print()

# Calculate
N_sb_ave = (N_sb_left + N_sb_right)/2
print(f"N sideband ave: {N_sb_ave}")
print()

N_sig = N_peak - N_sb_ave
print(f"N signal:     {N_sig}")


How do you think we did? Can you do better?

There are more sophisticated approaches to a data-driven background estimation, such as the 
[ABCD method](https://cms-opendata-guide.web.cern.ch/analysis/backgrounds/#abcd-method), 
which effectively extends the sidebands to a 2-dimensional plane of some kinematic variables. 

It's up to you to determine which approach is best for your analysis. 

# Simulation-based background estimation

Another approach is to use a 
[simulation based](https://cms-opendata-guide.web.cern.ch/analysis/backgrounds/#simulation) estimate
of your backgrounds and that is what we will use in our $Z'$ search. 

Basically this is making use of our simulation datasets that model standard model processes, the backgrounds
for our search for new physics. We add up all of these contributions and see if there is an excess that 
could be explained by our signal hypothesis. 

However, we need to take care that we are using our background Monte Carlo in a way that we can do an actual
comparison with the collision data. 

## Download the processed files

We're going to build upon the data processing we did in the [previous lesson on event selection](https://cms-opendata-workshop.github.io/workshop2024-lesson-event-selection/instructor/index.html).

We've processed all the files for you, using the exact same code as in that lesson. Let's get those files now. 

They are in a [tar file](https://en.wikipedia.org/wiki/Tar_(computing)) and when we untar it, it will 
create a subdirectory called `processed_files`.

In [ ]:
# Download the tar file
!wget https://github.com/cms-opendata-workshop/workshop2024-lesson-event-selection/raw/main/instructors/processed_files.tgz

# Untar it
# Verbose output
#!tar -zxvf processed_files.tgz

# Silent output
!tar -zxf processed_files.tgz

In [ ]:
# We can list our directory contents now
!ls -ltr

In [ ]:
# Take a look in the directory
!ls processed_files | tail -20

In [ ]:
# How many files are in the `processed_files` directory
!ls processed_files | wc -l

We processed 555 files out of all our datasets. As a reminder...

`collision` refers to the data and the other names refer to the signal MC and background MC samples. 

* Signal MC datasets
  * `signal_M2000`
* Background MC datasets
  * `ttsemilep`
  * `tthadronic`
  * `ttleptonic`
  * `Wjets`

What is in these files? We can open them and read them in as a pandas dataframe. 

In [ ]:
# Open one of the collision files
df_col = pd.read_csv('processed_files/OUTPUT_collision_0107961B-4308-F845-8F96-E14622BBA484.csv')

# Display the first 5 rows
df_col.head(5)

In [ ]:
# Open one of the simulation files
df_sim = pd.read_csv('processed_files/OUTPUT_tthadronic_009086DB-1E42-7545-9A35-1433EC89D04B.csv')

# Display the first 5 rows
df_sim.head(5)

555 files is a lot of files to keep track of! 

We're going to merge these together, so we wind up with 1 files per dataset. 

We have to keep track of the number of events we started with in a slightly more sophisticated fashion, 
but otherwise all we're doing is concatenating these files. 

In [ ]:
datasets = ['collision', 'signal_M2000', 'ttsemilep', 'tthadronic', 'ttleptonic', 'Wjets']

collision_datasets = datasets[0]
signal_datasets = datasets[1]
background_datasets = datasets[2:]

for dataset in datasets:
    
    N_gen = 0
    nevents = 0
    gw_pos = 0
    gw_neg = 0
    
    IS_COLLISION = False
    
    path = r'./' # use your path
    all_files = glob.glob(os.path.join(path , f"processed_files/OUTPUT_{dataset}*.csv"))
    
    print(f"Processing {len(all_files)} files in {dataset} ")

    list_of_dataframes = []

    for filename in all_files:
        
        if filename.find('collision') >= 0:
            IS_COLLISION = True
            
        df = pd.read_csv(filename, index_col=None, header=0)

        list_of_dataframes.append(df)

        if len(df) > 0:
            N_gen += df['N_gen'][0]
            nevents += df['nevents'][0]
            gw_pos += df['gw_pos'][0]
            gw_neg += df['gw_neg'][0]

    if IS_COLLISION:
        N_gen = -999
        gw_pos = -999
        gw_neg = -999

    df = pd.concat(list_of_dataframes, axis=0, ignore_index=True)
    df['N_gen'] = N_gen
    df['nevents'] = nevents
    df['gw_pos'] = gw_pos
    df['gw_neg'] = gw_neg
    
    df.to_csv(f'SUMMED_{dataset}.csv')


We now have a set of files called `SUMMED_ttsemilep.csv` for example. 

In [ ]:
!ls -ltr

We can see how many events remain in each dataset, just by getting the number of lines in each file. 

In [ ]:
!wc -l SUMMED*csv

In [ ]:
!head SUMMED_collision.csv

## Weights

We're going to be talking a lot about weights so lets explain what we mean and how these go into our histograms. 

A weight means that any individual event might count for more or less in our analysis. This could be because
of how they were generated or how they were processed. 

When we fill a histogram we can choose to weight each event, or some set of events by some amount. 

Here's an example with 2 events that go into a histogram with equal weight=1. 

In [ ]:
# Our data
x = [5, 10]

# The plot
plt.hist(x,bins=10, range=(0,10))
plt.ylim(0,5);

Now we weight it such that one entry has a weight of 0.5 and the other has a weight of 2. 

In [ ]:
# Our data
x = [5, 10]

# Our weights
weights = [0.5, 2]

# The plot
plt.hist(x, weights=weights, bins=10, range=(0,10))
plt.ylim(0,5);

We can also use this to save on computing power. For example, suppose
I am studying some particle that has a mass of 3.1 and a width of 0.1, if we model it
with a normal distribution (Gaussian). We can generate 100k events quite easily. 

In [ ]:
# Generate data
signal = np.random.normal(3.1, 0.1,100000)

# Plot the data
plt.hist(signal,bins=100, range=(2.6,3.6));

But suppose this is our background and we are interested in some other process that might
manifets around 3.4 or 3.5. Well, there are almost no entries out there and we don't want to generate a billion events, just to get good statistics out there. 

Instead, we can generate a uniform distribution of events and then calculate the *weight* for each event, 
if we assume that it comes from a normal distribution. The weight is just the normalized probability density
for that event coming from a normal distribution. 

In [ ]:
# A function to calculate the normalized probability density
def norm(mean, sigma, x):
    a = 1/(sigma*np.sqrt(2*np.pi))
    pdf = a*np.exp((-(x-mean)**2)/(2*(sigma**2)))
    return pdf

In [ ]:
# Our flat distribution of data
signal_flat = 2.6 + np.random.random(100000)

# The plot
plt.hist(signal_flat,bins=100, range=(2.6,3.6));

In [ ]:
# Use the function to calculate the probability density, which we store as the weight
# for each event

weights = norm(3.1,0.1, signal_flat)
weights



In [ ]:
# If we plot the weights, they don't seem to tell us anything
plt.hist(weights,bins=100);

In [ ]:
# But if we histogram the data and use the weights, we get our Gaussian back
plt.hist(signal_flat, weights=weights,bins=100, range=(2.6,3.6));

### Back to the data

Let's read in all our `.csv` files as dataframes and store them in a dictionary. 

We will also pull out the `N_gen` variable from the file and store that in the dictionary. 

In [ ]:
data = {}

for dataset in datasets:
    df = pd.read_csv(f'SUMMED_{dataset}.csv')
    
    N_gen = df['N_gen'][0]
    
    data[dataset] = {'df':df, 'N_gen': N_gen}

In [ ]:
data

In [ ]:
# We can access subsets of the dictionary to get individual datasets
data['ttsemilep']['df']

We can also compare our total number of events (either `N_gen` or `nevents`) with the
offical number on the Open Data portal. 

https://opendata.cern.ch/record/67993

Any small differences are usually attributable to the fact that we did not process every file perfectly. 

In [ ]:
data['ttsemilep']['N_gen']

We also need to fix something! 

A mistake was made when we combined all the `Wjets` files. Because some NanoAOD files had no events survive, 
those `.csv` files were empty and we never recorded the original number of generated events for those input files. 

To fix this, we will just set the value using the information from the Open Data Portal. 

https://opendata.cern.ch/record/69747

In [ ]:
# Mea culpa

data['Wjets']['N_gen'] = 80958227
data['Wjets']['df']['N_gen'] = 80958227

data['Wjets']['N_gen']

Now we can make a quick plot of all the different datasets!

Do any of them look different? Similar? 

In [ ]:
plt.figure(figsize=(12,7))

for i,dataset in enumerate(datasets):
    vals = data[dataset]['df']['mtt'].values
    
    plt.subplot(2,3,i+1)
    
    plt.hist(vals, bins=40, range=(0,4000), label=dataset)
    plt.xlabel(f'$M_{{t\overline{{t}}}}$ GeV/c$^2$', fontsize=14)
    
    plt.legend()
    
plt.tight_layout()

### Using `Hist`

We eventually want to compare the stacked background histograms with the collision data and you 
can do that with vanilla matplotlib or ROOT. 

However, we're going to use the [Hist](https://hist.readthedocs.io/en/latest/) library as it has some nice features. 

First, we make an empty histogram that allows us to save the data and put it into categories that depend on the dataset. 

It will have 40 bins and range from 0 to 4000 for the `mtt` variable. 

In [ ]:
h = Hist.new.Reg(40,0,4000, name='mtt', label=f'$M_{{t\overline{{t}}}}$ GeV/c$^2$')\
                .StrCat([], name="dataset", label="dataset", growth=True)\
                .Weight()

h

Now we can fill it quite easily! 

In [ ]:
# Loop over the datasets
for dataset in datasets:
    
    print(dataset)

    # Extract the mtt values from our dictionary
    vals = data[dataset]['df']['mtt'].values

    # Fill the histogram and label by dataset
    h.fill(mtt=vals, dataset=dataset) 

In [ ]:
# This is a hypersimple representation of what is in the histogram
h

Now that it is all in the histogram object, we can quickly stack the background datasets
and compare to our data. 

As a reminder we defined subsets of the datasets

In [ ]:
print("Collision datasets")
print(collision_datasets)
print()

print("Signal datasets")
print(signal_datasets)
print()

print("Background datasets")
print(background_datasets)
print()

print("All datasets")
print(datasets)

In [ ]:
plt.figure(figsize=(7,4))

# Stack the background datasets
h[:,background_datasets].stack('dataset')[:].project('mtt').plot(stack=True, histtype="fill")

# Overlay the collision datasets
h[:,collision_datasets].project('mtt').plot(histtype="errorbar", color='k', markersize=15, label='Collision')

# Increase the size of the x-axis label
plt.xlabel(xlabel=plt.gca().get_xlabel(), fontsize=18);

# Draw a legend
plt.legend()

plt.tight_layout()
plt.savefig('mtt_00.png')

Oh no!!! This looks *terrible*! The MC doesn't agree with the data at all!

Not surprising. We haven't weighted our MC samples appropriately. We tend to generate
more MC than you would expect to find in the data. So we need to take into account

* What is the integrated luminosity ($\mathcal{L}$) for the collision data we're working with?
    * 16400 inverse picobarns
* What is the cross section ($\sigma$) for these samples?
* How many events were generated for each sample?

We can then calculate a weight for each background sample. 

$$\textrm{weight} = \frac{\mathcal{L} \times \sigma}{N_{gen}}$$

For the signal, we will assume a cross section of 1 picobarn and then let the inference procedure tell us
what is the value that best described the data. 

For the `ttbar` backgrounds, we start with the production cross section for top/antitop production 
(831.76) and then scale by the probability of the W's decaying hadronically, leptonically, or one and the other. 

For the collision data, we just use a weight of 1.

We'll calculate this for each dataset and store it in our `data` dictionary. 

In [ ]:
integrated_luminosity = 16400 # inverse picobarns

ttbar_production_xsec = 831.76 # picobarns

data['Wjets']['xsec'] = 61526.7
data['ttsemilep']['xsec'] = ttbar_production_xsec*0.438
data['tthadronic']['xsec'] = ttbar_production_xsec*0.457
data['ttleptonic']['xsec'] = ttbar_production_xsec*0.105
data['signal_M2000']['xsec'] = 1 # Assume 1 pb


for dataset in [signal_datasets] + background_datasets:
    
    N_gen = data[dataset]['N_gen']
    
    xsec = data[dataset]['xsec']
    
    weight = integrated_luminosity * xsec / N_gen
    
    data[dataset]['weight'] = weight
    
data['collision']['weight'] = 1

In [ ]:
# Print them out 
for dataset in datasets:
    w = data[dataset]['weight']
    
    print(f'{dataset:20s}  {w:.2f}')

Let's redefine the Hist histograms and refill them, but this time we will use the weights we calculated. 

In [ ]:
h = Hist.new.Reg(40,0,4000, name='mtt', label=f'$M_{{t\overline{{t}}}}$ GeV/c$^2$')\
                .StrCat([], name="dataset", label="dataset", growth=True)\
                .Weight()

for dataset in datasets:
    
    print(dataset)

    vals = data[dataset]['df']['mtt'].values
    weight = data[dataset]['weight']


    h.fill(mtt=vals, dataset=dataset, weight=weight) 

Now we can make the same plot. 

In [ ]:
plt.figure(figsize=(7,4))

# Stack the background datasets
h[:,background_datasets].stack('dataset')[:].project('mtt').plot(stack=True, histtype="fill")

# Overlay the collision datasets
h[:,collision_datasets].project('mtt').plot(histtype="errorbar", color='k', markersize=15, label='Collision')

# Increase the size of the x-axis label
plt.xlabel(xlabel=plt.gca().get_xlabel(), fontsize=18);

# Draw a legend
plt.legend()

plt.tight_layout()
plt.savefig('mtt_01.png')

That's pretty good agreement for our zeroth-order reproduction of the analysis!

Let's overlay the signal to see where that would manifest. 

In [ ]:
plt.figure(figsize=(7,4))

# Stack the background datasets
h[:,background_datasets].stack('dataset')[:].project('mtt').plot(stack=True, histtype="fill")

# Overlay the collision datasets
h[:,collision_datasets].project('mtt').plot(histtype="errorbar", color='k', markersize=15, label='Collision')

# Overlay the signal hypothesis
h[:,signal_datasets].project('mtt').plot(color='y', lw=5, label=f"$Z'$ (M=2000 GeV/c$^2$")

# Increase the size of the x-axis label
plt.xlabel(xlabel=plt.gca().get_xlabel(), fontsize=18);

# Draw a legend
plt.legend()

plt.tight_layout()
plt.savefig('mtt_02.png')


From this we can see a few things

* Scaling the backgrounds appropriately does a pretty good job of modeling the collision data
* The background are primarily coming from the semileptonic decays of top/antitop pairs
* The signal peaks differently from the background
* There is no *obvious* signal